<a href="https://colab.research.google.com/github/acostabrandon/BSAN-6070/blob/main/brandon_acosta_ca02_draft_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#import necessary libraries
import numpy as np
import os
import numpy as np
from collections import Counter
from google.colab import drive #for google drive integration
from sklearn.naive_bayes import MultinomialNB #multinomial naive bayes model for text classification
from sklearn.metrics import accuracy_score #for testing accuracy

In [9]:
drive.mount('/content/drive')

Mounted at /content/drive


# Dictionary Function
This function starts by extracting the 3,000 most common words from all the emails in the training folder. It also removes any single-character words and non-alphabetical instances. The list that is outputed by this function will allow us to do feature extraction.

In [10]:
def make_Dictionary(root_dir):
  all_words = []
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
  for mail in emails:
    with open(mail) as m:
      for line in m:
        words = line.split()
        all_words += words
  dictionary = Counter(all_words)
  list_to_remove = list(dictionary)

  for item in list_to_remove:
    if item.isalpha() == False:
      del dictionary[item]
    elif len(item) == 1:
      del dictionary[item]
  dictionary = dictionary.most_common(3000)
  return dictionary

# Extract Features Function
This function reads all the emails in the training data set and converts each individual email into a "features matrix" using the top 3,000 words. It then counts each unique occurence (value counts) of each word and uploads it into the matrix. Then, based on the file name (such as the presence of 'spmsg'), it will return the number 1 for spam emails, and the number 0 for non-spam emails.

In [11]:
def extract_features(mail_dir):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000))
  train_labels = np.zeros(len(files))
  count = 1;
  docID = 0;
  for fil in files:
    with open(fil) as fi:
      for i, line in enumerate(fi):
        if i ==2:
          words = line.split()
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1;
        count = count + 1
      docID = docID + 1
  return features_matrix, train_labels

Paths using Google Drive to train folder and test folder.

In [12]:
TRAIN_DIR = '/content/drive/MyDrive/BSAN6070/train-mails'
TEST_DIR = '/content/drive/MyDrive/BSAN6070/test-mails'

# Training and Testing using Naive Bayes Model
The following block of code fills out the incomplete data from the one provided using the features matrix and train labels previously outputed by the two previous functions. Furthermore, it runs the model using Multinomial NB and tests the model for accuracy, providing a final accuracy score for the model.

In [13]:
#we start by using the make_Dictionary function to create the list of 3,000 most common words in the training data folder.
dictionary = make_Dictionary(TRAIN_DIR)

#we then use the extract_features function previously created using both the training and test dataset
print('Reading and processing emails from TRAIN and TEST folders.')
features_matrix, labels_train = extract_features(TRAIN_DIR)
test_features_matrix, labels_test = extract_features(TEST_DIR)

print('Training model using Multinomial Naive Bayes algorithm.')
model = MultinomialNB() #train the spam email classifier using Naive Bayes from SciKit Learn
model.fit(features_matrix, labels_train)

print('Training completed \n Testing trained model to predict test data labels.')
labels_predicted = model.predict(test_features_matrix) #this compares the predictions from the model to the test dataset matrix
accuracy = accuracy_score(labels_test, labels_predicted) #computes accuracy score

print('Completed classification of the test data... now printing Accuracy Score by comparing the predicted labels with the test labels:')
print(accuracy)

Reading and processing emails from TRAIN and TEST folders.
Training model using Multinomial Naive Bayes algorithm.
Training completed 
 Testing trained model to predict test data labels.
Completed classification of the test data... now printing Accuracy Score by comparing the predicted labels with the test labels:
0.9615384615384616
